# AI: Machine Learning w/ Imbalanced Classes <a class = "anchor" id = "top"></a>

#### - Zan

---

In [ ]:
# Read data
from google.colab import files
uploaded = files.upload()

Saving stroke_data.csv to stroke_data (1).csv


In [ ]:
# Install packages
#import sys
#!{sys.executable} -m pip install ann_visualizer
#!{sys.executable} -m pip install graphviz
#!{sys.executable} -m pip install keras
#!{sys.executable} -m pip install tensorflow

In [ ]:
# Import libraries
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense
import numpy
from ann_visualizer.visualize import ann_viz
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, classification_report, roc_curve, plot_roc_curve, auc, precision_recall_curve, plot_precision_recall_curve, average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import io

In [26]:
# Get
df = pd.read_csv(io.BytesIO(uploaded['stroke_data.csv'])).drop('id', axis = 1)
df.head(n = 10)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
6,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
7,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
8,Female,59.0,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
9,Female,78.0,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1


In [27]:
# Encode categorical columns and split for training
cats = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
data = data_refinery(df, cats, 'stroke')

In [28]:
data.xtrain.head(n = 10)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
3956,1,36.0,0,0,0,2,1,200.68,25.8,0
574,1,18.0,0,0,0,2,1,112.17,31.7,0
4013,1,44.0,1,0,1,2,0,91.28,26.5,2
3081,0,72.0,0,0,1,3,0,104.04,34.7,1
4201,1,43.0,0,0,1,2,0,207.37,29.5,1
1412,1,82.0,1,0,1,2,0,227.28,33.3,2
972,0,16.0,0,0,0,2,1,87.98,22.4,2
4526,1,60.0,0,0,1,2,1,80.67,33.5,0
3792,0,74.0,0,0,1,2,1,130.37,26.3,0
2275,0,82.0,0,0,1,3,0,117.75,29.8,2


In [29]:
# Oversampling with SMOTE
sm = SMOTE(random_state = 100)

smote_xtrain, smote_ytrain = sm.fit_resample(data.xtrain, data.ytrain.ravel())
smote_xval, smote_yval = sm.fit_resample(data.xval, data.yval.ravel())

In [30]:
# Function to count values
def counter(x, condition):
    
    x1 = [i for i in x if i == condition]
    
    return len(x1)

print('Overall Target Distribution: ')
print(df['stroke'].value_counts())
print()
print('Distribution for Machine Learning: \n')
print('Training- \n')
print(f'Negative: {counter(data.ytrain, 0)}')
print(f'Positive: {counter(data.ytrain, 1)}')
print('\nValidation- \n')
print(f'Negative: {counter(data.yval, 0)}')
print(f'Positive: {counter(data.yval, 1)}')
print('\nSMOTE Target Distribution:\n')
print('Training- \n')
print(f'Negative, {counter(smote_ytrain, 0)}')
print(f'Positive, {counter(smote_ytrain, 1)}')
print('\nValidation- \n')
print(f'Negative, {counter(smote_yval, 0)}')
print(f'Positive, {counter(smote_yval, 1)}')

Overall Target Distribution: 
0    4700
1     209
Name: stroke, dtype: int64

Distribution for Machine Learning: 

Training- 

Negative: 3752
Positive: 175

Validation- 

Negative: 948
Positive: 34

SMOTE Target Distribution:

Training- 

Negative, 3752
Positive, 3752

Validation- 

Negative, 948
Positive, 948


#### Ground-Truth Model:

In [31]:
pipeline = modeling_pipeline(data.xtrain, data.ytrain, data.xval, data.yval)
pipeline.execute_pipeline_flow()

results = pd.DataFrame()

for k, v in pipeline.scores.items():
    
    results = results.append({'model': k,
                              'acc': v[0],
                              'kfold_acc': v[1],
                              'roc': v[2],
                              'precision': v[3],
                              'recall': v[4],
                              'f1': v[5]}, ignore_index = True)

LogisticRegression :

 [[947   1]
 [ 32   2]]

Accuracy Score:  0.97
K-Fold Validation Mean Accuracy: 96.44%
Standard Deviation: 0.68
ROC AUC Score: 0.53
Precision: 0.67
Recall: 0.06
F1: 0.11
--------------------------------------

SVM :

 [[948   0]
 [ 34   0]]

Accuracy Score:  0.97
K-Fold Validation Mean Accuracy: 96.54%
Standard Deviation: 0.49
ROC AUC Score: 0.50
Precision: 0.00
Recall: 0.00
F1: 0.00
--------------------------------------

KNeighbors :

 [[939   9]
 [ 34   0]]

Accuracy Score:  0.96
K-Fold Validation Mean Accuracy: 96.33%
Standard Deviation: 0.67
ROC AUC Score: 0.50
Precision: 0.00
Recall: 0.00
F1: 0.00
--------------------------------------

GaussianNB :

 [[834 114]
 [ 19  15]]

Accuracy Score:  0.86
K-Fold Validation Mean Accuracy: 88.80%
Standard Deviation: 2.46
ROC AUC Score: 0.66
Precision: 0.12
Recall: 0.44
F1: 0.18
--------------------------------------

BernoulliNB :

 [[936  12]
 [ 32   2]]

Accuracy Score:  0.96
K-Fold Validation Mean Accuracy: 95.93%
S

#### SMOTE Model:

In [32]:
smote_pipeline = modeling_pipeline(smote_xtrain, smote_ytrain, smote_xval, smote_yval)
smote_pipeline.execute_pipeline_flow()

smote_results = pd.DataFrame()

for k, v in smote_pipeline.scores.items():
    
    smote_results = smote_results.append({'model': k,
                                          'acc': v[0],
                                          'kfold_acc': v[1],
                                          'roc': v[2],
                                          'precision': v[3],
                                          'recall': v[4],
                                          'f1': v[5]}, ignore_index = True)

LogisticRegression :

 [[711 237]
 [268 680]]

Accuracy Score:  0.73
K-Fold Validation Mean Accuracy: 81.49%
Standard Deviation: 1.85
ROC AUC Score: 0.73
Precision: 0.74
Recall: 0.72
F1: 0.73
--------------------------------------

SVM :

 [[664 284]
 [237 711]]

Accuracy Score:  0.73
K-Fold Validation Mean Accuracy: 80.27%
Standard Deviation: 1.81
ROC AUC Score: 0.73
Precision: 0.71
Recall: 0.75
F1: 0.73
--------------------------------------

KNeighbors :

 [[784 164]
 [434 514]]

Accuracy Score:  0.68
K-Fold Validation Mean Accuracy: 90.40%
Standard Deviation: 2.10
ROC AUC Score: 0.68
Precision: 0.76
Recall: 0.54
F1: 0.63
--------------------------------------

GaussianNB :

 [[705 243]
 [263 685]]

Accuracy Score:  0.73
K-Fold Validation Mean Accuracy: 79.06%
Standard Deviation: 1.92
ROC AUC Score: 0.73
Precision: 0.74
Recall: 0.72
F1: 0.73
--------------------------------------

BernoulliNB :

 [[792 156]
 [259 689]]

Accuracy Score:  0.78
K-Fold Validation Mean Accuracy: 81.27%
S

In [ ]:
print(results.sort_values('f1', ascending = False)[['model', 'precision', 'recall', 'roc']])

                model  precision    recall       roc
3          GaussianNB   0.116279  0.441176  0.660462
5        DecisionTree   0.121951  0.147059  0.554542
0  LogisticRegression   0.666667  0.058824  0.528884
4         BernoulliNB   0.142857  0.058824  0.523083
6        RandomForest   0.500000  0.029412  0.514178
7             XGBoost   0.111111  0.029412  0.510486
1                 SVM   0.000000  0.000000  0.500000
2          KNeighbors   0.000000  0.000000  0.495253


In [ ]:
print(smote_results.sort_values('f1', ascending = False)[['model', 'precision', 'recall', 'roc']])

                model  precision    recall       roc
7             XGBoost   0.958564  0.732068  0.850211
0  LogisticRegression   0.789095  0.809072  0.796414
3          GaussianNB   0.745918  0.771097  0.754219
1                 SVM   0.718053  0.746835  0.726793
6        RandomForest   0.915447  0.593882  0.769515
4         BernoulliNB   0.632083  0.835443  0.674578
2          KNeighbors   0.777135  0.566456  0.702004
5        DecisionTree   0.858377  0.524262  0.718882


#### Neural Network Visualization:

In [33]:
# Init NN
model = Sequential()
model.add(Dense(12, input_dim = len(data.xtrain.columns), activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(smote_xtrain, smote_ytrain, epochs = 10, batch_size = 1000)
scores = model.evaluate(smote_xval, smote_yval)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))

Epoch 1/10
8/8 [==============================] - 1s 3ms/step - loss: 15.6549 - accuracy: 0.4953
Epoch 2/10
8/8 [==============================] - 0s 2ms/step - loss: 12.0234 - accuracy: 0.4980
Epoch 3/10
8/8 [==============================] - 0s 2ms/step - loss: 8.5772 - accuracy: 0.4987
Epoch 4/10
8/8 [==============================] - 0s 3ms/step - loss: 5.2742 - accuracy: 0.4985
Epoch 5/10
8/8 [==============================] - 0s 2ms/step - loss: 2.3187 - accuracy: 0.5093
Epoch 6/10
8/8 [==============================] - 0s 2ms/step - loss: 1.2100 - accuracy: 0.5032
Epoch 7/10
8/8 [==============================] - 0s 4ms/step - loss: 1.3679 - accuracy: 0.4694
Epoch 8/10
8/8 [==============================] - 0s 2ms/step - loss: 1.1388 - accuracy: 0.5058
Epoch 9/10
8/8 [==============================] - 0s 3ms/step - loss: 0.8877 - accuracy: 0.5676
Epoch 10/10
60/60 [==============================] - 0s 1ms/step - loss: 0.6169 - accuracy: 0.6472

accuracy: 64.72%


In [34]:
# Evaluate
preds = model.predict(smote_xval)
preds = [1 if x >= .5 else 0 for x in preds]

cm = confusion_matrix(smote_yval, preds)  
accuracies = accuracy_score(smote_yval, preds)
roc = roc_auc_score(smote_yval, preds)  
precision = precision_score(smote_yval, preds) 
recall = recall_score(smote_yval, preds) 
f1 = f1_score(smote_yval, preds)  

print('Outputs for Neural Network- ')
print(cm)
print('Accuracy Score: ', round(accuracy_score(smote_yval, preds), 2))
print('ROC AUC Score: {:.2f}'.format(roc))
print('Precision: {:.2f}'.format(precision))
print('Recall: {:.2f}'.format(recall))
print('F1: {:.2f}'.format(f1))

Outputs for Neural Network- 
[[703 245]
 [424 524]]
Accuracy Score:  0.65
ROC AUC Score: 0.65
Precision: 0.68
Recall: 0.55
F1: 0.61


In [37]:
ann_viz(model, title = "Architecture for NN to Predict Stroke", filename = "diabetes_network")